In [1]:
import pandas as pd

# Load dataset
data = pd.read_excel('Listings.xlsx')
data.head()

,Year,Make,Model,Price,Mileage,Location,Additional,Fuel Type,Car Type,Scraping Time,Source,Car Build
0,2023,Hyundai,Tucson,39860,575,VIC,*Excl. Govt. Charges,Petrol,2.0L SUV FWD,1711775988,Drive,SUV
1,2021,Toyota,RAV4,48990,26346,NSW,*Excl. Govt. Charges,Hybrid,2.5L SUV FWD Hybrid,1711775989,Drive,SUV
2,2022,LDV,T60,34996,24444,WA,*Excl. Govt. Charges,Diesel,LUXE 2.0L Diesel Dual Cab Ute 4XD,1711775989,Drive,UTE
3,2020,Mazda,CX-3,25490,46351,NSW,*Excl. Govt. Charges,Petrol,Sport 2.0L SUV FWD,1711775992,Drive,SUV
4,2022,Hyundai,i30,55950,6700,VIC,*Excl. Govt. Charges,Petrol,2.0L Hatchback FWD,1711775992,Drive,OTHER


In [2]:
# Read the Excel file
df = pd.read_excel('MSRPs.xlsx')

# Select only the specified columns
msrp_data = df[['Year', 'Make', 'Model', 'Fuel Type', 'Car Type','MSRP']]

msrp_data.head()

,Year,Make,Model,Fuel Type,Car Type,MSRP
0,2019,Hyundai,IONIQ,Hybrid,Elite 1.6L FWD Hybrid,33990
1,2019,Hyundai,IONIQ,Electric,Premium FWD,48990
2,2019,Hyundai,Kona,Petrol,2.0L SUV FWD,23500
3,2019,Hyundai,Kona,Petrol,1.6L SUV 4WD,27000
4,2019,Hyundai,Kona,Electric,Highlander SUV FWD,64640


In [3]:
# Define the columns to check for duplicates
columns_to_check = ['Year', 'Make', 'Model', 'Price', 'Mileage', 'Location']

before_removal = len(data)

# Drop duplicates based on the specified columns
data_unique = data.drop_duplicates(subset=columns_to_check)

after_removal = len(data_unique)

removed = before_removal - after_removal

print(removed)

data_unique.head()

172


,Year,Make,Model,Price,Mileage,Location,Additional,Fuel Type,Car Type,Scraping Time,Source,Car Build
0,2023,Hyundai,Tucson,39860,575,VIC,*Excl. Govt. Charges,Petrol,2.0L SUV FWD,1711775988,Drive,SUV
1,2021,Toyota,RAV4,48990,26346,NSW,*Excl. Govt. Charges,Hybrid,2.5L SUV FWD Hybrid,1711775989,Drive,SUV
2,2022,LDV,T60,34996,24444,WA,*Excl. Govt. Charges,Diesel,LUXE 2.0L Diesel Dual Cab Ute 4XD,1711775989,Drive,UTE
3,2020,Mazda,CX-3,25490,46351,NSW,*Excl. Govt. Charges,Petrol,Sport 2.0L SUV FWD,1711775992,Drive,SUV
4,2022,Hyundai,i30,55950,6700,VIC,*Excl. Govt. Charges,Petrol,2.0L Hatchback FWD,1711775992,Drive,OTHER


In [4]:
# Perform the merge
merged_data_og = pd.merge(
    data_unique,
    msrp_data[['Year', 'Make', 'Model', 'Fuel Type', 'Car Type', 'MSRP']],
    on=['Year', 'Make', 'Model', 'Fuel Type', 'Car Type'],
    how='inner'
)

# Shuffle the DataFrame
shuffled_df = merged_data_og.sample(frac=1, random_state=1).reset_index(drop=True)

# Group by the specified columns and limit each group to 20 rows
group_columns = ['Year', 'Make', 'Model', 'Fuel Type', 'Car Type']
limited_df = shuffled_df.groupby(group_columns).head(20)

merged_data = limited_df

# Display the merged data
merged_data.head()

,Year,Make,Model,Price,Mileage,Location,Additional,Fuel Type,Car Type,Scraping Time,Source,Car Build,MSRP
0,2020,Toyota,RAV4,42725,44962,NSW,Drive Away,Hybrid,2.5L SUV 4WD Hybrid,1719026736,Drive,SUV,46290
1,2022,MG,ZS,18490,24507,QLD,Drive Away,Petrol,1.5L SUV FWD,1712976756,Drive,SUV,21990
2,2022,BMW,iX3,74990,28566,WA,*Excl. Govt. Charges,Electric,SUV RWD,1720826607,Drive,SUV,114900
3,2019,Nissan,X-TRAIL,23956,65059,SA,*Excl. Govt. Charges,Petrol,2.5L SUV FWD,1718427980,Drive,SUV,29990
4,2019,Toyota,Corolla,30890,15515,NSW,Drive Away,Hybrid,Hybrid 1.8L Hatchback FWD Hybrid,1711776584,Drive,OTHER,26335


In [5]:
import datetime

# Calculate the Retention Rate
merged_data['Retention Rate'] = merged_data['Price'] / merged_data['MSRP']

# Remove rows where Retention Rate is greater than 2
merged_data = merged_data[merged_data['Retention Rate'] <= 2]

# Calculate the Age of the cars
current_year = datetime.datetime.now().year
merged_data['Age'] = current_year - merged_data['Year']

# Filter for ages greater than 0 and mileage less tha 500,000 
merged_data = merged_data[merged_data['Age'] > 0]
merged_data = merged_data[merged_data['Mileage'] < 500000]


merged_data['Fuel Type Adjusted'] = merged_data.apply(lambda x:
                                        ('Electric' if x['Fuel Type'] == 'Electric' else
                                         ('Hybrid' if x['Fuel Type'] == 'Hybrid' else 'Conventional')),
                                        axis=1)


merged_data.head()

#merged_data.to_csv("merged_data.csv", index=False)

C:\Users\Gunne\AppData\Local\Temp\ipykernel_42264\2092954812.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['Retention Rate'] = merged_data['Price'] / merged_data['MSRP']
C:\Users\Gunne\AppData\Local\Temp\ipykernel_42264\2092954812.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['Age'] = current_year - merged_data['Year']


,Year,Make,Model,Price,Mileage,Location,Additional,Fuel Type,Car Type,Scraping Time,Source,Car Build,MSRP,Retention Rate,Age,Fuel Type Adjusted
0,2020,Toyota,RAV4,42725,44962,NSW,Drive Away,Hybrid,2.5L SUV 4WD Hybrid,1719026736,Drive,SUV,46290,0.922986,4,Hybrid
1,2022,MG,ZS,18490,24507,QLD,Drive Away,Petrol,1.5L SUV FWD,1712976756,Drive,SUV,21990,0.840837,2,Conventional
2,2022,BMW,iX3,74990,28566,WA,*Excl. Govt. Charges,Electric,SUV RWD,1720826607,Drive,SUV,114900,0.652654,2,Electric
3,2019,Nissan,X-TRAIL,23956,65059,SA,*Excl. Govt. Charges,Petrol,2.5L SUV FWD,1718427980,Drive,SUV,29990,0.798800,5,Conventional
4,2019,Toyota,Corolla,30890,15515,NSW,Drive Away,Hybrid,Hybrid 1.8L Hatchback FWD Hybrid,1711776584,Drive,OTHER,26335,1.172964,5,Hybrid
